In [1]:
!pip install PyPDF2 pdf2image pytesseract nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y poppler-utils
!apt-get install -y poppler-utils tesseract-ocr

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [75.2 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/mai

In [71]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [74]:
import os
import random
import re
from typing import List, Dict, Tuple, Any
import PyPDF2
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from google.colab import files
import io

In [75]:
class QGenerator:
    def __init__(self):
        """Initializing"""
        self.question_templates = {
            'factual': [
                "What is {}?",
                "Who is {}?",
                "When did {} occur?",
                "Where is {} located?",
                "How does {} work?",
                "What are the components of {}?",
                "What is the definition of {}?",
                "What are the characteristics of {}?",
            ],
            'conceptual': [
                "Why is {} important?",
                "How does {} relate to {}?",
                "What is the significance of {}?",
                "What are the implications of {}?",
                "How would you explain {} to someone new to the subject?",
                "What is the purpose of {}?",
                "How does {} affect {}?",
            ],
            'analytical': [
                "Compare and contrast {} and {}.",
                "What would happen if {} changed?",
                "What evidence supports {}?",
                "How would you analyze {}?",
                "What conclusions can be drawn from {}?",
                "What are the strengths and weaknesses of {}?",
                "How would you evaluate the effectiveness of {}?",
            ],
            'application': [
                "How can {} be applied to solve {}?",
                "What is an example of {} in the real world?",
                "How would you implement {} in practice?",
                "What would be a use case for {}?",
                "How could {} be used to improve {}?",
            ]
        }


        self.important_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

        self.cover_page_patterns = [
            r'cover\s*page',
            r'title\s*page',
            r'copyright',
            r'all\s*rights\s*reserved',
            r'table\s*of\s*contents',
            r'index',
            r'glossary',
            r'references',
            r'bibliography',
            r'appendix',
        ]

        self.noise_patterns = [
            r'^\d+$',
            r'^\s*page\s*\d+\s*$',
            r'^\s*\d+\s*of\s*\d+\s*$',
            r'^\s*chapter\s*\d+\s*$',
            r'^\s*section\s*\d+\.*\d*\s*$',
            r'^\s*www\..+\..+\s*$',
            r'^\s*https?://.+\s*$',
            r'^\s*[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\s*$',
        ]

    def _is_cover_page(self, text: str) -> bool:
        for pattern in self.cover_page_patterns:
            if re.search(pattern, text.lower()):
                return True

        if len(text.strip()) < 200:
            return True

        sentences = sent_tokenize(text)
        if len(sentences) < 3:
            return True

        return False

    def _clean_noise(self, text: str) -> str:
        lines = text.split('\n')
        cleaned_lines = []

        for line in lines:
            if not line.strip():
                continue

            if any(re.search(pattern, line.strip()) for pattern in self.noise_patterns):
                continue

            if len(line.strip()) < 5:
                continue

            cleaned_lines.append(line)

        return '\n'.join(cleaned_lines)

    def _calculate_content_relevance(self, text: str) -> float:
        sentences = sent_tokenize(text)
        sentence_count = len(sentences)

        words = word_tokenize(text.lower())
        unique_words = set(words)
        unique_word_count = len(unique_words)

        if len(words) > 0:
            info_density = unique_word_count / len(words)
        else:
            info_density = 0

        relevance_score = sentence_count * info_density

        return relevance_score

    def _extract_text_from_pdf(self, pdf_path: str) -> List[Dict[str, Any]]:
        page_texts = []
        try:
            with open(pdf_path, "rb") as file:
                reader = PyPDF2.PdfReader(file)

                start_page = 0
                if len(reader.pages) > 1:
                    first_page_text = reader.pages[0].extract_text() or ""
                    if self._is_cover_page(first_page_text):
                        start_page = 1
                        print("Skipping cover page...")

                for page_num, page in enumerate(reader.pages[start_page:], start_page + 1):
                    page_text = page.extract_text() or ""


                    if not page_text.strip() or self._is_cover_page(page_text):
                        continue

                    cleaned_text = self._clean_noise(page_text)

                    relevance_score = self._calculate_content_relevance(cleaned_text)

                    if relevance_score > 5:
                        page_texts.append({
                            'text': cleaned_text,
                            'page': page_num,
                            'source': 'direct',
                            'relevance': relevance_score
                        })
        except Exception as e:
            print(f"Error extracting text directly: {str(e)}")

        return page_texts

    def _extract_text_from_pdf_images(self, pdf_path: str) -> List[Dict[str, Any]]:
        page_texts = []
        try:
            images = convert_from_path(pdf_path)

            start_page = 0
            if len(images) > 1:
                first_page_text = pytesseract.image_to_string(images[0])
                if self._is_cover_page(first_page_text):
                    start_page = 1
                    print("Skipping cover page image...")

            for i, image in enumerate(images[start_page:], start_page + 1):
                page_text = pytesseract.image_to_string(image)

                if not page_text.strip() or self._is_cover_page(page_text):
                    continue

                cleaned_text = self._clean_noise(page_text)

                relevance_score = self._calculate_content_relevance(cleaned_text)

                if relevance_score > 5:
                    page_texts.append({
                        'text': cleaned_text,
                        'page': i,
                        'source': 'image',
                        'relevance': relevance_score
                    })
        except Exception as e:
            print(f"Error extracting text from images: {str(e)}")

        return page_texts

    def _clean_text(self, text: str) -> str:
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s.,;:!?()-]', '', text)
        return text.strip()

    def _extract_important_phrases(self, page_texts: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        important_phrases = []

        for page_data in page_texts:
            text = page_data['text']
            page_num = page_data['page']

            sentences = sent_tokenize(text)
            for sentence in sentences:
                words = word_tokenize(sentence)
                tagged_words = pos_tag(words)

                current_phrase = []
                for word, tag in tagged_words:
                    if tag in self.important_pos_tags and word.lower() not in stopwords.words('english'):
                        current_phrase.append(word)
                    elif current_phrase:
                        if len(current_phrase) >= 1:
                            phrase = ' '.join(current_phrase)
                            important_phrases.append({
                                'phrase': phrase,
                                'page': page_num,
                                'context': sentence
                            })
                        current_phrase = []

                if current_phrase and len(current_phrase) >= 1:
                    phrase = ' '.join(current_phrase)
                    important_phrases.append({
                        'phrase': phrase,
                        'page': page_num,
                        'context': sentence
                    })

        unique_phrases = {}
        for item in important_phrases:
            phrase = item['phrase']
            if phrase not in unique_phrases or len(phrase) > len(unique_phrases[phrase]['phrase']):
                unique_phrases[phrase] = item

        result = list(unique_phrases.values())
        result.sort(key=lambda x: len(x['phrase']), reverse=True)

        return result[:50]

    def _extract_key_sentences(self, page_texts: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        key_sentences = []

        importance_indicators = [
            'important', 'significant', 'key', 'main', 'primary', 'essential', 'critical',
            'fundamental', 'crucial', 'vital', 'major', 'central', 'core', 'notable',
            'define', 'definition', 'means', 'refer', 'example', 'instance', 'illustrate',
            'demonstrate', 'show', 'prove', 'evidence', 'research', 'study', 'analysis',
            'conclude', 'summary', 'therefore', 'thus', 'hence', 'consequently'
        ]

        for page_data in page_texts:
            text = page_data['text']
            page_num = page_data['page']

            sentences = sent_tokenize(text)
            for sentence in sentences:
                if any(indicator in sentence.lower() for indicator in importance_indicators):
                    key_sentences.append({
                        'sentence': sentence,
                        'page': page_num
                    })

                elif re.search(r'\b[A-Z][a-z]+ (is|are|refers to|means|can be defined as)\b', sentence):
                    key_sentences.append({
                        'sentence': sentence,
                        'page': page_num
                    })

        return key_sentences

    def _find_answer_for_phrase(self, phrase: str, page_texts: List[Dict[str, Any]], context: str = None) -> Dict[str, Any]:
        for page_data in page_texts:
            text = page_data['text']
            page_num = page_data['page']

            definition_patterns = [
                f"{phrase} is ",
                f"{phrase} are ",
                f"{phrase} refers to ",
                f"{phrase} means ",
                f"{phrase} can be defined as ",
                f"define {phrase} as ",
                f"{phrase} is defined as "
            ]

            sentences = sent_tokenize(text)
            for sentence in sentences:
                if any(pattern.lower() in sentence.lower() for pattern in definition_patterns):
                    return {
                        'answer': sentence,
                        'page': page_num
                    }

        if context:
            for page_data in page_texts:
                if context in page_data['text']:
                    return {
                        'answer': context,
                        'page': page_data['page']
                    }

        relevant_sentences = []
        for page_data in page_texts:
            text = page_data['text']
            page_num = page_data['page']

            sentences = sent_tokenize(text)
            for sentence in sentences:
                if phrase.lower() in sentence.lower():
                    relevant_sentences.append({
                        'sentence': sentence,
                        'page': page_num,
                        'relevance': len(sentence)
                    })

        relevant_sentences.sort(key=lambda x: x['relevance'])

        if relevant_sentences:
            best_match = relevant_sentences[0]
            return {
                'answer': best_match['sentence'],
                'page': best_match['page']
            }

        return {
            'answer': f"Information about {phrase} can be found in the document.",
            'page': 'unknown'
        }

    def _generate_factual_questions_with_answers(self, phrases: List[Dict[str, Any]], page_texts: List[Dict[str, Any]], num: int) -> List[Dict[str, Any]]:
        questions = []
        templates = self.question_templates['factual']

        phrases_copy = phrases.copy()
        random.shuffle(phrases_copy)

        for phrase_data in phrases_copy[:num]:
            phrase = phrase_data['phrase']
            context = phrase_data.get('context')
            template = random.choice(templates)
            question = template.format(phrase)

            answer_data = self._find_answer_for_phrase(phrase, page_texts, context)

            questions.append({
                'question': question,
                'answer': answer_data['answer'],
                'page': answer_data['page'],
                'type': 'factual'
            })

        return questions

    def _generate_conceptual_questions_with_answers(self, phrases: List[Dict[str, Any]], page_texts: List[Dict[str, Any]], num: int) -> List[Dict[str, Any]]:
        questions = []
        templates = self.question_templates['conceptual']

        phrases_copy = phrases.copy()
        random.shuffle(phrases_copy)

        for i, template in enumerate(random.sample(templates, min(num, len(templates)))):
            if "{}" in template and "{}" in template[template.index("{}") + 2:]:
                if len(phrases_copy) >= 2:
                    phrase_data1 = phrases_copy.pop(0)
                    phrase_data2 = phrases_copy.pop(0)
                    phrase1 = phrase_data1['phrase']
                    phrase2 = phrase_data2['phrase']
                    question = template.format(phrase1, phrase2)

                    both_phrases = f"{phrase1} and {phrase2}"
                    answer_data = self._find_answer_for_phrase(both_phrases, page_texts)

                    if answer_data['page'] == 'unknown':
                        answer_data = self._find_answer_for_phrase(phrase1, page_texts, phrase_data1.get('context'))

                    questions.append({
                        'question': question,
                        'answer': answer_data['answer'],
                        'page': answer_data['page'],
                        'type': 'conceptual'
                    })
            else:
                if phrases_copy:
                    phrase_data = phrases_copy.pop(0)
                    phrase = phrase_data['phrase']
                    question = template.format(phrase)

                    answer_data = self._find_answer_for_phrase(phrase, page_texts, phrase_data.get('context'))

                    questions.append({
                        'question': question,
                        'answer': answer_data['answer'],
                        'page': answer_data['page'],
                        'type': 'conceptual'
                    })

        return questions

    def _generate_analytical_questions_with_answers(self, phrases: List[Dict[str, Any]], key_sentences: List[Dict[str, Any]], page_texts: List[Dict[str, Any]], num: int) -> List[Dict[str, Any]]:
        questions = []
        templates = self.question_templates['analytical']

        phrases_copy = phrases.copy()
        random.shuffle(phrases_copy)

        for i, template in enumerate(random.sample(templates, min(num, len(templates)))):
            if "{}" in template and "{}" in template[template.index("{}") + 2:]:
                if len(phrases_copy) >= 2:
                    phrase_data1 = phrases_copy.pop(0)
                    phrase_data2 = phrases_copy.pop(0)
                    phrase1 = phrase_data1['phrase']
                    phrase2 = phrase_data2['phrase']
                    question = template.format(phrase1, phrase2)

                    answer1 = self._find_answer_for_phrase(phrase1, page_texts, phrase_data1.get('context'))
                    answer2 = self._find_answer_for_phrase(phrase2, page_texts, phrase_data2.get('context'))

                    combined_answer = f"Regarding {phrase1}: {answer1['answer']} Regarding {phrase2}: {answer2['answer']}"

                    questions.append({
                        'question': question,
                        'answer': combined_answer,
                        'page': f"{answer1['page']} and {answer2['page']}",
                        'type': 'analytical'
                    })
            else:
                if phrases_copy:
                    phrase_data = phrases_copy.pop(0)
                    phrase = phrase_data['phrase']
                    question = template.format(phrase)

                    answer_data = self._find_answer_for_phrase(phrase, page_texts, phrase_data.get('context'))

                    questions.append({
                        'question': question,
                        'answer': answer_data['answer'],
                        'page': answer_data['page'],
                        'type': 'analytical'
                    })

        if key_sentences and len(questions) < num:
            for sentence_data in random.sample(key_sentences, min(num - len(questions), len(key_sentences))):
                sentence = sentence_data['sentence']
                page = sentence_data['page']

                question = f"Analyze the following statement: '{sentence}'"

                context_answer = self._find_context_for_sentence(sentence, page_texts, page)

                questions.append({
                    'question': question,
                    'answer': context_answer['answer'],
                    'page': context_answer['page'],
                    'type': 'analytical'
                })

        return questions

    def _find_context_for_sentence(self, sentence: str, page_texts: List[Dict[str, Any]], target_page: int) -> Dict[str, Any]:
        for page_data in page_texts:
            if page_data['page'] == target_page:
                text = page_data['text']

                sentences = sent_tokenize(text)
                for i, s in enumerate(sentences):
                    if sentence in s:
                        context = []
                        if i > 0:
                            context.append(sentences[i-1])
                        context.append(s)
                        if i < len(sentences) - 1:
                            context.append(sentences[i+1])

                        return {
                            'answer': ' '.join(context),
                            'page': target_page
                        }

        return {
            'answer': sentence,
            'page': target_page
        }

    def _generate_application_questions_with_answers(self, phrases: List[Dict[str, Any]], page_texts: List[Dict[str, Any]], num: int) -> List[Dict[str, Any]]:
        questions = []
        templates = self.question_templates['application']

        phrases_copy = phrases.copy()
        random.shuffle(phrases_copy)

        for i, template in enumerate(random.sample(templates, min(num, len(templates)))):
            if "{}" in template and "{}" in template[template.index("{}") + 2:]:
                if len(phrases_copy) >= 2:
                    phrase_data1 = phrases_copy.pop(0)
                    phrase_data2 = phrases_copy.pop(0)
                    phrase1 = phrase_data1['phrase']
                    phrase2 = phrase_data2['phrase']
                    question = template.format(phrase1, phrase2)

                    answer1 = self._find_answer_for_phrase(phrase1, page_texts, phrase_data1.get('context'))
                    answer2 = self._find_answer_for_phrase(phrase2, page_texts, phrase_data2.get('context'))

                    combined_answer = f"Based on the document: Regarding {phrase1}: {answer1['answer']} Regarding {phrase2}: {answer2['answer']}"

                    questions.append({
                        'question': question,
                        'answer': combined_answer,
                        'page': f"{answer1['page']} and {answer2['page']}",
                        'type': 'application'
                    })
            else:
                if phrases_copy:
                    phrase_data = phrases_copy.pop(0)
                    phrase = phrase_data['phrase']
                    question = template.format(phrase)

                    answer_data = self._find_answer_for_phrase(phrase, page_texts, phrase_data.get('context'))

                    answer = f"Based on the document: {answer_data['answer']}"

                    questions.append({
                        'question': question,
                        'answer': answer,
                        'page': answer_data['page'],
                        'type': 'application'
                    })

        return questions

    def generate_questions_with_answers_from_pdf(self, pdf_path: str, num_questions: int = 10) -> List[Dict[str, Any]]:
        print("Extracting text from PDF and filtering out less necessary content...")

        direct_page_texts = self._extract_text_from_pdf(pdf_path)

        image_page_texts = self._extract_text_from_pdf_images(pdf_path)

        all_page_texts = direct_page_texts + image_page_texts

        all_page_texts.sort(key=lambda x: x.get('relevance', 0), reverse=True)

        most_relevant_pages = all_page_texts[:min(len(all_page_texts), 20)]

        for page_data in most_relevant_pages:
            page_data['text'] = self._clean_text(page_data['text'])

        total_text = ''.join(page_data['text'] for page_data in most_relevant_pages)
        if not total_text or len(total_text) < 50:
            return [{
                'question': "The PDF doesn't contain enough extractable text to generate questions.",
                'answer': "Please try a different PDF with more text content.",
                'page': "N/A",
                'type': 'error'
            }]

        print(f"Successfully extracted content from {len(most_relevant_pages)} relevant pages.")

        important_phrases = self._extract_important_phrases(most_relevant_pages)
        key_sentences = self._extract_key_sentences(most_relevant_pages)

        print(f"Identified {len(important_phrases)} important phrases and {len(key_sentences)} key sentences.")

        num_factual = max(1, int(num_questions * 0.4))
        num_conceptual = max(1, int(num_questions * 0.3))
        num_analytical = max(1, int(num_questions * 0.2))
        num_application = max(1, num_questions - num_factual - num_conceptual - num_analytical)

        print("Generating questions and finding answers...")

        factual_questions = self._generate_factual_questions_with_answers(important_phrases, most_relevant_pages, num_factual)
        conceptual_questions = self._generate_conceptual_questions_with_answers(important_phrases, most_relevant_pages, num_conceptual)
        analytical_questions = self._generate_analytical_questions_with_answers(important_phrases, key_sentences, most_relevant_pages, num_analytical)
        application_questions = self._generate_application_questions_with_answers(important_phrases, most_relevant_pages, num_application)

        all_questions = factual_questions + conceptual_questions + analytical_questions + application_questions

        if len(all_questions) < num_questions and important_phrases:
            additional_needed = num_questions - len(all_questions)
            additional_questions = self._generate_factual_questions_with_answers(important_phrases, most_relevant_pages, additional_needed)
            all_questions.extend(additional_questions)

        random.shuffle(all_questions)

        print(f"Successfully generated {len(all_questions)} questions with answers.")

        return all_questions[:num_questions]

def generate_questions_with_answers_from_uploaded_pdf(num_questions=10):
    print("Please upload a PDF file...")
    uploaded = files.upload()

    if not uploaded:
        print("No file was uploaded.")
        return

    file_name = list(uploaded.keys())[0]

    print(f"\nProcessing {file_name}...")
    print("This may take a few minutes depending on the PDF size and complexity.")

    generator = QGenerator()

    questions_with_answers = generator.generate_questions_with_answers_from_pdf(file_name, num_questions)

    print(f"\nGenerated {len(questions_with_answers)} questions with answers from {file_name}:")
    for i, qa in enumerate(questions_with_answers, 1):
        print(f"\n{i}. {qa['question']}")
        print(f"   Answer: {qa['answer']}")
        print(f"   Page: {qa['page']}")
        print(f"   Type: {qa['type']}")

    return questions_with_answers

In [76]:
num_questions = 30

questions_with_answers = generate_questions_with_answers_from_uploaded_pdf(num_questions)

Please upload a PDF file...


Saving Research .pdf to Research  (5).pdf

Processing Research  (5).pdf...
This may take a few minutes depending on the PDF size and complexity.
Extracting text from PDF and filtering out less necessary content...
Skipping cover page...
Skipping cover page image...
Successfully extracted content from 4 relevant pages.
Identified 50 important phrases and 22 key sentences.
Generating questions and finding answers...
Successfully generated 30 questions with answers.

Generated 30 questions with answers from Research  (5).pdf:

1. How can Understanding eigenvalues be applied to solve fundamental concepts?
   Answer: Based on the document: Regarding Understanding eigenvalues: Mathematically, for a square matrix A, a vector v is an eigenvector and A is A-v 2 eV the eigenvalue if: Understanding eigenvalues and eigenvectors allows us to grasp the behavior of linear transformations and systems, making them essential in fields like computer graphics, machine learning, data science, and image pro

In [77]:
if questions_with_answers:
    with open('questions_and_answers.txt', 'w') as f:
        for i, qa in enumerate(questions_with_answers, 1):
            f.write(f"{i}. {qa['question']}\n")
            f.write(f"   Answer: {qa['answer']}\n")
            f.write(f"   Page: {qa['page']}\n")
            f.write(f"   Type: {qa['type']}\n\n")

    files.download('questions_and_answers.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>